In [ ]:
import os
import glob
import numpy as np
import cv2
from PIL import Image
import math

import PIL
from rembg import remove
from preprocessing.image_processing import remove_background_from_folder, remove_background_recursive

## Background Removal
- Remove the background of the image and save image as RGBA without a background

In [ ]:
target_folder = "example_data/Meshfleet_Eval/Results_002"
remove_background_recursive(target_folder)

## Front View Detection

In [ ]:
import torch
from preprocessing.image_processing import align_views, load_dino, load_dino_model, get_dino_transform, release_gpu
# from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

"""
vlm_model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
vlm_processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
vlm_model.to(device)
"""

dino_model = load_dino(device)
dino_transform = get_dino_transform(224)

In [ ]:
gt_folder = "example_data/Meshfleet_Eval/Ground_Truth"
generated_folder = "example_data/Meshfleet_Eval/Results_002"
prompt = "Is this a front view or a rear view?"
df = align_views(generated_folder, detection_mode="dino", dino_model=dino_model, dino_transform=dino_transform, ground_truth_parent=gt_folder, device=device)

## Equal Scaling
- Adapt both images to equal scaling 
- Width of original image is set and generated images width is set to the same value while keeping the aspect ration fixed
- Objects are maximized in the image

### Equal Scaling for multiple folders
- Folder Structure is:
      ground_truth_parent/
          <object_id_1>/
              000.png, 001.png, ..., 011.png
          <object_id_2>/
              ...
      generated_parent/
          <object_id_1>/
              000.png, 001.png, ..., 011.png
          <object_id_2>/
              ...

In [ ]:
from preprocessing.image_processing import process_equal_scaling, process_equal_scaling_structure

process_equal_scaling_structure(
    ground_truth_parent="example_data/Meshfleet_Eval/Ground_Truth",
    generated_parent="example_data/Meshfleet_Eval/Results_002",
    canvas_size=(768,768),
    fill_ratio=0.9
)

### Equal Scaling for a folder pair with no subfolders

In [ ]:
original_folder = "example_data/examples_08082025/Car_Sample_2/original"
generated_folder = "example_data/examples_08082025/Car_Sample_2/generated_side"

out_original_folder = "example_data/examples_08082025/Car_Sample_2/scaled/original"
out_generated_folder = "example_data/examples_08082025/Car_Sample_2/scaled/generated_side"

process_equal_scaling(original_folder, generated_folder,
                        out_original_folder, out_generated_folder,
                        canvas_size=(768,768), fill_ratio=0.95)


## Concatenate images into grid
- Just for visualization

In [ ]:
from preprocessing.image_processing import concatenate_images

output_file_name = "concatenated1.png"
concatenate_images(
    "/home/q613871/Projects/Trellis_Playground/TRELLIS/assets/Meshfleet_Eval/Results_006", 
    output_image=output_file_name, 
    rows = 2, 
    target_identifier = "000.png",
    crop_width_percent=30,
    crop_height_percent=50)

## Manual Image Reordering
- Can be used to manually restructure a subfolder --> Given index will be set as the new 000.png

In [ ]:
from preprocessing.image_processing import reorder_images_in_folder

target_folder = "assets/Meshfleet_Eval/019d5adeba1cc38c1dbc550b0306d008272765d13c49dec2e08a4d1276c1cca1"
reorder_images_in_folder(target_folder, 6)

## Viewpoint Alignment
- Due to the front view detection it can happen that the viewpoints are swapped which leads to the vehicle being "rotated" in the wrong direction
- With this function the user can swap the direction of the rotation by swapping the images

In [ ]:
from preprocessing.image_processing import restructure_images
restructure_images("example_data/Meshfleet_Eval/Results_002")